# Notebook with test code

---

## Imports

### Standard library imports

In [1]:
import yaml

import sys

from datetime import (date, datetime)

import os

import pickle

import re

### Third party libraries

In [2]:
import boto3

import pandas as pd

### Local application imports

In [3]:
sys.path.append("../")

In [4]:
%load_ext autoreload
%autoreload 2

from src.etl.ingesta_almacenamiento import (
    
    ## Functions
    get_client,
    ingesta_inicial,
    ingesta_consecutiva,
    get_s3_resource,
    request_data_to_API,
    get_s3_credentials,
    
)


from src.utils.utils import (

    get_api_token

)


from src.utils.data_dict import (
    data_dict
)

---

## AWS base examples

### S3

#### Initial configuration

##### Resource

##### Client

In [ ]:
s3_creds = get_s3_credentials("../conf/local/credentials.yaml")

session = boto3.Session(
    aws_access_key_id=s3_creds['aws_access_key_id'],
    aws_secret_access_key=s3_creds['aws_secret_access_key']
)
s3 = session.client('s3')

#### Connecting to existing bucket and viewing contents

In [ ]:
bucket = "data-product-architecture-equipo-9"
key ="ingestion/consecutive"

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket, Prefix=key)['Contents']
objects

#### Reading pickle from S3

In [ ]:
obj_path = [file["Key"] for file in objects if "2021-03-18" in file["Key"]][0]
obj_path

In [ ]:
response = s3.get_object(
    Bucket=bucket,
    Key=obj_path
)
response

In [ ]:
body = response["Body"].read()

---

In [ ]:
pickle.loads(pickle.loads(body))

#### Creating bucket

In [ ]:
bucket_name = "comdline-test-bucket-rob"

s3.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={'LocationConstraint': 'us-west-2'},
    ACL="private"
)

#### Uploading file to bucket

In [ ]:
file_to_upload = "../../admin/test_file3_for_s3.txt"
file_name = file_to_upload.split(sep="/")[-1]

In [ ]:
# s3.upload_file(file_to_upload, bucket_name, file_name)

s3.meta.client.upload_file(file_to_upload, bucket_name, file_name)

In [ ]:
[obj for obj in s3.Bucket(bucket_name).objects.all()]

#### Downloading files from bucket

In [ ]:
path_to_download = "/Users/rp_mbp/Documents/ReposRob_RobPer/ITAMmcd/semestre_2/Arquitectura_Prod_Dat/Venv_ArqPD/repos/dpa_2021/_robdir/" + file_name

s3.meta.client.download_file(bucket_name, file_name, path_to_download)

#### Deleting bucket files

In [ ]:
s3.Object(bucket_name, file_name).delete()

---

#### Deleting bucket

## Downloading and storing Chicago data

#### Interacting with API

In [12]:
dataset_id = "4ijn-s7e5"

In [13]:
token = get_api_token("../conf/local/credentials.yaml")
token

'5HfsId12lhMMzSlYANoAq451w'

In [14]:
client = get_client(token)
client

In [15]:
most_rec_date = '2021-04-10'

In [16]:
soql_query = "inspection_date >= '{}'".format(most_rec_date)
soql_query

"inspection_date >= '2021-04-10'"

In [17]:
x = client.get(dataset_id, 
               limit=10,
               where=soql_query
              )

In [18]:
x

[{'inspection_id': '2500305',
  'dba_name': '7-ELEVEN #38948H',
  'aka_name': '7-ELEVEN',
  'license_': '2738590',
  'facility_type': 'Grocery Store',
  'risk': 'Risk 2 (Medium)',
  'address': '2366 N DAMEN AVE ',
  'city': 'CHICAGO',
  'state': 'IL',
  'zip': '60647',
  'inspection_date': '2021-04-16T00:00:00.000',
  'inspection_type': 'Canvass',
  'results': 'Out of Business',
  'latitude': '41.924737806379355',
  'longitude': '-87.67802914773546',
  'location': {'latitude': '-87.67802914773546',
   'longitude': '41.924737806379355'}},
 {'inspection_id': '2500264',
  'dba_name': 'UNCLE REMUS',
  'aka_name': 'UNCLE REMUS PLUS GUILTY PLEASURES',
  'license_': '2642353',
  'facility_type': 'Restaurant',
  'risk': 'Risk 1 (High)',
  'address': '4731 W MADISON ST ',
  'city': 'CHICAGO',
  'state': 'IL',
  'zip': '60644',
  'inspection_date': '2021-04-15T00:00:00.000',
  'inspection_type': 'Complaint Re-Inspection',
  'results': 'Pass',
  'latitude': '41.88040118054544',
  'longitude': '-8

In [19]:
len(x)

10

In [21]:
xx = pd.DataFrame(x)
xx

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,latitude,longitude,location,violations
0,2500305,7-ELEVEN #38948H,7-ELEVEN,2738590,Grocery Store,Risk 2 (Medium),2366 N DAMEN AVE,CHICAGO,IL,60647,2021-04-16T00:00:00.000,Canvass,Out of Business,41.924737806379355,-87.67802914773546,"{'latitude': '-87.67802914773546', 'longitude'...",NaN
1,2500264,UNCLE REMUS,UNCLE REMUS PLUS GUILTY PLEASURES,2642353,Restaurant,Risk 1 (High),4731 W MADISON ST,CHICAGO,IL,60644,2021-04-15T00:00:00.000,Complaint Re-Inspection,Pass,41.88040118054544,-87.74408951751022,"{'latitude': '-87.74408951751022', 'longitude'...",NaN
2,2500241,DUNKIN DONUTS,DUNKIN DONUTS (T2 F9),2630933,Restaurant,Risk 2 (Medium),11601 W TOUHY AVE,CHICAGO,IL,60666,2021-04-14T00:00:00.000,Canvass,Pass,42.008536400868735,-87.91442843927047,"{'latitude': '-87.91442843927047', 'longitude'...",36. THERMOMETERS PROVIDED & ACCURATE - Comment...
3,2500239,CRAGIN PUBLIC COFFEE,CRAGIN PUBLIC COFFEE,2780866,Restaurant,Risk 2 (Medium),3150 N CENTRAL AVE,CHICAGO,IL,60634,2021-04-14T00:00:00.000,License,Fail,41.93825446024883,-87.76663539027273,"{'latitude': '-87.76663539027273', 'longitude'...","3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."
4,2498204,IRISH AMERICAN HERITAGE CENTER,IRISH AMERICAN HERITAGE CENTER,40543,Restaurant,Risk 1 (High),4626 N KNOX AVE,CHICAGO,IL,60630,2021-04-14T00:00:00.000,Non-Inspection,No Entry,41.965242580480265,-87.74411673243358,"{'latitude': '-87.74411673243358', 'longitude'...",NaN
5,2498188,EL TEXCAL RESTAURANT,EL TEXCAL RESTAURANT,2652423,Restaurant,Risk 1 (High),3700 W LAWRENCE AVE,CHICAGO,IL,60625,2021-04-14T00:00:00.000,Canvass Re-Inspection,Pass,41.96843111302068,-87.72076005083134,"{'latitude': '-87.72076005083134', 'longitude'...",NaN
6,2498186,HOLIDAY INN CHICAGO O'HARE,HOLIDAY INN/BAR LOUIE,1518913,Restaurant,Risk 1 (High),5615 N CUMBERLAND AVE,CHICAGO,IL,60631,2021-04-13T00:00:00.000,Complaint,Fail,41.98142816417737,-87.83643430209348,"{'latitude': '-87.83643430209348', 'longitude'...",2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...
7,2498184,BLUE BOTTLE COFFEE,BLUE BOTTLE COFFEE,2776561,NaN,Risk 2 (Medium),313 W WOLF POINT PLZ,CHICAGO,IL,60654,2021-04-13T00:00:00.000,License,Not Ready,41.88780561739225,-87.63682820571593,"{'latitude': '-87.63682820571593', 'longitude'...",NaN
8,2498202,TATAS TACOS,TATAS TACOS,2781098,Restaurant,Risk 1 (High),2826 N LINCOLN AVE,CHICAGO,IL,60657,2021-04-13T00:00:00.000,License,Pass,41.93310149870225,-87.65968295657315,"{'latitude': '-87.65968295657315', 'longitude'...",NaN
9,2500253,RealGood Stuff Co,Real Good Stuff,2780825,Restaurant,Risk 3 (Low),2275 N LINCOLN AVE,CHICAGO,IL,60614,2021-04-13T00:00:00.000,License Re-Inspection,Pass w/ Conditions,41.92361620972698,-87.64599112233753,"{'latitude': '-87.64599112233753', 'longitude'...",NaN


In [24]:
str(xx.shape)

'(10, 17)'

##### Using "dumps"

In [ ]:
xx = pickle.dumps(x)

In [ ]:
xx

In [ ]:
pickle.loads(xx)

##### Using "dump"

In [ ]:
pkl_store_loc = !pwd
pkl_store_loc = pkl_store_loc[0] + "/prueba_pickle.pkl"

In [ ]:
pkl_store_loc

In [ ]:
pickle.dump(x, open(pkl_store_loc, 'wb'))

In [ ]:
pickle.load(open(pkl_store_loc, "rb"))

##### Storing data in S3

In [ ]:
pkl_store_loc = !pwd
pkl_store_loc = pkl_store_loc[0] + "/prueba_pickle.pkl"
pkl_store_loc

In [ ]:
## Storing file locally
pickle.dump(x, open(pkl_store_loc, "wb"))

In [ ]:
## Loading file from local and saving as variable
xx = pickle.dumps(pickle.load(open(pkl_store_loc, "rb")))

In [ ]:
## Saving pickle in s3
s3.put_object(
    Bucket=bucket_name,
    Key="test_pickle.pkl",
    Body=xx
)

##### Downloading data from s3 and unpickling

In [ ]:
res_xx = s3.get_object(
    Bucket=bucket_name,
    Key="test_pickle.pkl"
)
res_xx

In [ ]:
body = res_xx["Body"].read()

In [ ]:
pickle.loads(body)

#### Uploading info

#### Checking aws s3 contents

In [5]:
s3 = get_s3_resource()

FileNotFoundError: Couldnt load the file

In [ ]:
cont_ingest_path

In [ ]:
objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=cont_ingest_path)['Contents']
objects

In [ ]:
objects.append(
 {'Key': 'ingestion/consecutive/consecutive-inspections-2021-02-22.pkl',
  'LastModified': "datetime.datetime(2021, 2, 22, 2, 53, 54, tzinfo=tzutc())",
  'ETag': '"79cff7864a646f1dfd4d51b4e732a226"',
  'Size': 1152660,
  'StorageClass': 'STANDARD'}
)

objects.append(
 {'Key': 'ingestion/consecutive/consecutive-inspections-2021-02-15.pkl',
  'LastModified': "datetime.datetime(2021, 2, 22, 2, 53, 54, tzinfo=tzutc())",
  'ETag': '"79cff7864a646f1dfd4d51b4e732a226"',
  'Size': 1152660,
  'StorageClass': 'STANDARD'}
)

objects.append(
 {'Key': 'ingestion/consecutive/consecutive-inspections-2021-02-10.pkl',
  'LastModified': "datetime.datetime(2021, 2, 22, 2, 53, 54, tzinfo=tzutc())",
  'ETag': '"79cff7864a646f1dfd4d51b4e732a226"',
  'Size': 1152660,
  'StorageClass': 'STANDARD'}
)

In [ ]:
rex = str(cont_dat_prefix) + "(.*).pkl"

lx = [datetime.strptime(re.search(rex, obj["Key"]).group(1), '%Y-%m-%d') for obj in objects if cont_dat_prefix in obj["Key"]]
most_rec_date = datetime.strftime(max(lx), '%Y-%m-%d')
most_rec_date

In [ ]:
s3x = pickle.loads(s3.get_object(Bucket=bucket_name, Key=(cont_ingest_path + "consecutive-inspections-2021-02-21.pkl"))['Body'].read())

In [ ]:
dfx = pd.DataFrame(s3x)
dfx

In [ ]:
dfx["inspection_date"] = pd.to_datetime(dfx["inspection_date"])

In [ ]:
print("max date: ", max(dfx["inspection_date"]))
print("min date: ", min(dfx["inspection_date"]))

In [ ]:
dfx.groupby(pd.Grouper(key="inspection_date")).count()[["inspection_id"]]

---

## Creating local directories for temporal data

In [ ]:
base_path = "../src/pipeline/luigi/ingestion_tmp/"

### Find most recent ingestion

#### Case 1: There is previous consecutive downloads

In [ ]:
lyrs = [ydir[-4:] for ydir in os.listdir(base_path + "consecutive") if "YEAR=" in ydir]
lyrs

In [ ]:
mr_yr = max(lyrs)
mr_yr

In [ ]:
lmths = [mdir[-2:] for mdir in os.listdir(base_path + "consecutive" + "/" + "YEAR=" + mr_yr) if "MONTH=" in mdir]
lmths

In [ ]:
mr_mth = max(lmths)
mr_mth

In [ ]:
lings = [ing for ing in os.listdir(base_path + "consecutive" + "/" + "YEAR=" + mr_yr + "/" + "MONTH=" + mr_mth)]
lings

In [ ]:
regex = "consecutive_inspections_" + "(.*).pkl"

In [ ]:
min([re.search(regex, ing).group(1) for ing in lings])

In [ ]:
lx = ['.DS_Store', 'consecutive_inspections_2021-03-05.pkl', 'consecutive_inspections_2021-03-10.pkl']
lx

In [ ]:
cont_dat_prefix = "consecutive_inspections_"

In [ ]:
regex = cont_dat_prefix + "(.*).pkl"

In [ ]:
lx

In [ ]:
lings

In [ ]:
most_recent_ing = max([re.search(regex, ing).group(1) for ing in lx if ".pkl" in ing])
most_recent_ing

#### Additional notes

In [ ]:
x = date.today().strftime('%Y-%m-%d')

In [ ]:
x[5:7]

---

## Creating .csv files with metadata

### Working with previous metadata pickle

In [ ]:
pkl_meta = "../results/metadata/transform_metadata.pkl"

In [ ]:
dfx = pickle.load(open(pkl_meta, "rb"))
dfx

In [ ]:
mx = [dfx.index[0]]
mx

In [ ]:
[mx.append(val) for val in dfx.iloc[0, :]]

In [ ]:
mx

In [ ]:
## Overwriting csv file from a dataframe (no header)
def write_csv_from_df(df, filepath, filename):
    
    ## Extracting df contents as list
    mdata_list = [df.index[0]]
    [mdata_list.append(val) for val in df.iloc[0, :]]
    
    ## Creating and writing csv file with extracted list
    with open(filepath + filename, mode="w") as metadata_file:
        metadata_writer = csv.writer(metadata_file, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        metadata_writer.writerow(mdata_list)

In [ ]:
filepath = "metadata_test_dir/"

In [ ]:
filename = "transformation_metadata_2.csv"

In [ ]:
write_csv_from_df(dfx, filepath, filename)

In [ ]:
dfx

### Creating dummy metadata

#### Line 1

In [ ]:
meta_1 = str(datetime.now())

In [ ]:
meta_2 = str(10)

In [ ]:
meta_3 = "carnegie"

In [ ]:
meta_comp = ",".join([meta_1, meta_2, meta_3])
meta_comp

In [ ]:
import csv
with open("transformation_metadata.csv", mode="w") as metadata_file:
    metadata_writer = csv.writer(metadata_file, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
    metadata_writer.writerow([meta_1, meta_2, meta_3])

## Saving dictionary of dataframes as pickle

### Creating dictionary of dataframes

In [9]:
dfx1 = {
    "col1_df1": list(range(1, 11)),
    "col2_df1": list("htkdlekdjc"),
}
dfx1 = pd.DataFrame.from_dict(dfx1)
dfx1

,col1_df1,col2_df1
0,1,h
1,2,t
2,3,k
3,4,d
4,5,l
5,6,e
6,7,k
7,8,d
8,9,j
9,10,c


In [10]:
dfx2 = {
    "col1_df2": list(range(11, 21)),
    "col2_df2": list("jfldurytgc"),
}
dfx2 = pd.DataFrame.from_dict(dfx2)
dfx2

,col1_df2,col2_df2
0,11,j
1,12,f
2,13,l
3,14,d
4,15,u
5,16,r
6,17,y
7,18,t
8,19,g
9,20,c


In [11]:
df_dict = {
    "dfx1": dfx1,
    "dfx2": dfx2
}
df_dict

{'dfx1':    col1_df1 col2_df1
 0         1        h
 1         2        t
 2         3        k
 3         4        d
 4         5        l
 5         6        e
 6         7        k
 7         8        d
 8         9        j
 9        10        c,
 'dfx2':    col1_df2 col2_df2
 0        11        j
 1        12        f
 2        13        l
 3        14        d
 4        15        u
 5        16        r
 6        17        y
 7        18        t
 8        19        g
 9        20        c}

In [13]:
df_dict['dfx2']

,col1_df2,col2_df2
0,11,j
1,12,f
2,13,l
3,14,d
4,15,u
5,16,r
6,17,y
7,18,t
8,19,g
9,20,c


### Storing dictionary as pickle variable and unpickling

In [16]:
df_dict_pkl = pickle.dumps(df_dict)

In [19]:
df_dict_x = pickle.loads(df_dict_pkl)

In [21]:
df_dict_x['dfx2']

,col1_df2,col2_df2
0,11,j
1,12,f
2,13,l
3,14,d
4,15,u
5,16,r
6,17,y
7,18,t
8,19,g
9,20,c


### Storing dictionary as pickle in local memory and unpickling

In [24]:
pkl_file = "../data/pickles/test_df_dict_pkl.pkl"

In [28]:
pickle.dump(df_dict, open(pkl_file, "wb"))

In [33]:
df_dict_x = pickle.load(open(pkl_file, "rb"))

In [35]:
df_dict_x['dfx2']

,col1_df2,col2_df2
0,11,j
1,12,f
2,13,l
3,14,d
4,15,u
5,16,r
6,17,y
7,18,t
8,19,g
9,20,c


---

## *Notes*

---
---